# Demonstration notebook for processing raw RINEX data
In this Notebook, we process some example RINEX files to demonstrate gnssvod.

In [1]:
import gnssvod as gv

## gv.preprocess()
The main pre-processing function is preprocess(). This function  will do several things
- It will read RINEX observation files as pandas data frames
- It can aggregate the raw data to a lower temporal rate if specified.
- It will by default download orbit and clock files for the corresponding days from the GSSC ESA server
- From the orbit and clock files, it will calculate azimuth and elevation for each measurement
- It can save each processed file as a netcdf file in the outputdir folder or return the results as a dictionary

### specifying input files
The function exclusively reads RINEX observation files. Such files typically end with the extension '.yyO' where yy is the last two digit of the year. The function can be used to process a single file, a group of files, or several groups of files corresponding to several receivers, as shown in the examples below. All of this is done by specifying a pattern as the first argument to the function.

### specifying output destinations
Results are saved to a NetCDF file when an output directory is specified and/or returned as a dictionary when "outputresult=True" is passed.

Let's read a single file using the example data to begin with

In [2]:
pattern = {'Dav2_Twr':'data_RINEX2.11/Dav2_Twr/rinex/Reach_Dav2_Twr-raw_202104282106.21O'}
result = gv.preprocess(pattern,outputresult=True)

data_RINEX2.11/Dav2_Twr/rinex/Reach_Dav2_Twr-raw_202104282106.21O exists | Reading...
Observation file  data_RINEX2.11/Dav2_Twr/rinex/Reach_Dav2_Twr-raw_202104282106.21O  is read in 2.69 seconds.
Processing 112382 individual observations
Calculating Azimuth and Elevation
This file does not exist: GFZ0MGXRAP_20211180000_01D_05M_ORB.SP3
Downloading: GFZ0MGXRAP_20211180000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20211180000_01D_05M_ORB.SP3.gz: 0.98MB [00:01, 769kB/s]                             


 | Download completed for GFZ0MGXRAP_20211180000_01D_05M_ORB.SP3.gz
GFZ0MGXRAP_20211180000_01D_05M_ORB.SP3 file is read in 2.12 seconds
This file does not exist: GFZ0MGXRAP_20211190000_01D_05M_ORB.SP3
Downloading: GFZ0MGXRAP_20211190000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20211190000_01D_05M_ORB.SP3.gz: 0.98MB [00:03, 336kB/s]                             


 | Download completed for GFZ0MGXRAP_20211190000_01D_05M_ORB.SP3.gz
GFZ0MGXRAP_20211190000_01D_05M_ORB.SP3 file is read in 3.76 seconds
This file does not exist: GFZ0MGXRAP_20211180000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20211180000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20211180000_01D_30S_CLK.CLK.gz: 4.54MB [00:02, 2.34MB/s]                            


 | Download completed for GFZ0MGXRAP_20211180000_01D_30S_CLK.CLK.gz
GFZ0MGXRAP_20211180000_01D_30S_CLK.CLK file is read in 2.01 seconds
This file does not exist: GFZ0MGXRAP_20211190000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20211190000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20211190000_01D_30S_CLK.CLK.gz: 4.55MB [00:02, 2.34MB/s]                            


 | Download completed for GFZ0MGXRAP_20211190000_01D_30S_CLK.CLK.gz
GFZ0MGXRAP_20211190000_01D_30S_CLK.CLK file is read in 1.91 seconds
SP3 interpolation is done in 59.36 seconds


The default logs should indicate how many observations were read in the file. If this is the first time you run the script, it also shows some orbit files were downloaded.

If you process very recent data (less than 3 days old), it could be that the orbit and clock files are not available on the ESA server yet and there would then be an error.

The result returned by the function is a dictionary providing lists of Observation objects.

In [3]:
result

{'Dav2_Twr': [<gnssvod.io.io.Observation at 0x7fd179bd26a0>]}

Since we processed one file, there is only one Observation object in the list. Let us access this first and unique item.

In [4]:
obs = result['Dav2_Twr'][0]
obs

Observation objects are custom classes introduced in the `gnsspy` package by Mustafa Serkan Işık and Volkan Özbey. A significant number of base functions in `gnssvod` are based on gnsspy.

Observation objects contain the following properties
- obs.filename          = the name of the source file
- obs.epoch             = a datetime indicate the day at the start of the record
- obs.observation       = a pandas data frame containing all measurements
- obs.approx_position   = the approximate receiver position as provided in the RINEX file [X,Y,Z]
- obs.receiver_type     = the receiver type if provided in the RINEX file
- obs.antenna_type      = the antenna type if provided in the RINEX file
- obs.interval          = the measurement frequency in seconds
- obs.receiver_clock    = the receiver clock if provided in the RINEX file
- obs.version           = the version of the RINEX file
- obs.observation_types = the observation types reported as columns in obs.observation

Let's just look at the data..

In [5]:
obs.observation

C1            L1        D1    S1  \
Epoch               SV                                                
2021-04-28 21:07:08 G01  2.300005e+07  1.208661e+08 -2545.050  47.0   
                    G03  2.257959e+07  1.186567e+08  -528.921  47.0   
                    G04  2.373431e+07  1.247247e+08  1971.157  45.0   
                    G09  2.620530e+07  1.377098e+08  3155.798  40.0   
                    G17  2.423057e+07  1.273325e+08  -403.684  49.0   
...                               ...           ...       ...   ...   
2021-04-28 22:07:07 C16           NaN  2.282536e+08 -1695.972  36.0   
                    C27           NaN  1.432542e+08  1694.596  44.0   
                    C28           NaN  1.326840e+08 -1191.327  49.0   
                    S23  4.111082e+07  2.160388e+08  -516.361  45.0   
                    S36  4.078565e+07  2.143301e+08  -517.120  42.0   

                                   C2            L2        D2    S2  \
Epoch               SV                                                
2021-04-28 21:07:08 G01  2.300006e+07  9.418140e+07 -1983.223  44.0   
                    G03  2.257959e+07  9.245973e+07  -412.197  42.0   
                    G04  2.373431e+07  9.718807e+07  1535.983  42.0   
                    G09  2.620531e+07  1.073063e+08  2459.033  40.0   
                    G17  2.423058e+07  9.922014e+07  -314.543  40.0   
...                               ...           ...       ...   ...   
2021-04-28 22:07:07 C16  4.383370e+07  2.282536e+08 -1695.972  36.0   
                    C27  2.751047e+07  1.432542e+08  1694.596  44.0   
                    C28  2.548056e+07  1.326840e+08 -1191.327  49.0   
                    S23           NaN           NaN       NaN   NaN   
                    S36           NaN           NaN       NaN   NaN   

                                   C7  L7        D7    S7     Azimuth  \
Epoch               SV                                                  
2021-04-28 21:07:08 G01           NaN NaN       NaN   NaN  132.372864   
                    G03           NaN NaN       NaN   NaN  -16.744176   
                    G04           NaN NaN       NaN   NaN -169.822432   
                    G09           NaN NaN       NaN   NaN -150.328510   
                    G17           NaN NaN       NaN   NaN  -81.696037   
...                               ...  ..       ...   ...         ...   
2021-04-28 22:07:07 C16  4.383372e+07 NaN -1311.197  31.0   41.172558   
                    C27           NaN NaN       NaN   NaN  -42.395746   
                    C28           NaN NaN       NaN   NaN  -99.551780   
                    S23           NaN NaN       NaN   NaN         NaN   
                    S36           NaN NaN       NaN   NaN         NaN   

                         Elevation  
Epoch               SV              
2021-04-28 21:07:08 G01  59.902810  
                    G03  79.170432  
                    G04  50.693499  
                    G09  19.288991  
                    G17  43.789922  
...                            ...  
2021-04-28 22:07:07 C16   7.343129  
                    C27  20.938009  
                    C28  49.140759  
                    S23        NaN  
                    S36        NaN  

[112382 rows x 14 columns]

The pandas data frame has a MultIndex that contains both Epoch and SV as indices. The columns correspond to:
- C# = Pseudorange from the receiver to the satellite, in meters
- L# = Carrier phase, in cycles
- D# = Doppler, in Hz
- S# = Carrier to noise density C/N$_0$, in dB (receiver-dependent)

And the numbers (S1, S2, etc. ) indicate the corresponding GNSS frequency

The azimuth and elevation of the satellite with respect to the receiver are expressed in degrees. Computation speed for the azimuth and elevation can vary according to your hardware. Most of the time is spent interpolating the orbit parameters to the time stamps of each measurement. This is why it is sometimes useful to aggregate high frequency data (here one measurement per second) to for instance one measurement each 15 seconds.

### resampling

We can pass "interval='15S'" to resample the data during the preprocessing. The returned data will be smaller and the calculation of the azimuths and elevations (reported as "SP3 interpolation") will be faster.

In [6]:
pattern = {'Dav2_Twr':'data_RINEX2.11/Dav2_Twr/rinex/Reach_Dav2_Twr-raw_202104282106.21O'}
result = gv.preprocess(pattern,interval='15S',outputresult=True)
# and show data frame
result['Dav2_Twr'][0].observation

data_RINEX2.11/Dav2_Twr/rinex/Reach_Dav2_Twr-raw_202104282106.21O exists | Reading...
Observation file  data_RINEX2.11/Dav2_Twr/rinex/Reach_Dav2_Twr-raw_202104282106.21O  is read in 2.63 seconds.
Processing 112382 individual observations
Calculating Azimuth and Elevation
GFZ0MGXRAP_20211180000_01D_05M_ORB.SP3 exists | Reading...
GFZ0MGXRAP_20211180000_01D_05M_ORB.SP3 file is read in 0.28 seconds
GFZ0MGXRAP_20211190000_01D_05M_ORB.SP3 exists | Reading...
GFZ0MGXRAP_20211190000_01D_05M_ORB.SP3 file is read in 0.28 seconds
GFZ0MGXRAP_20211180000_01D_30S_CLK.CLK exists | Reading...
GFZ0MGXRAP_20211180000_01D_30S_CLK.CLK file is read in 1.32 seconds
GFZ0MGXRAP_20211190000_01D_30S_CLK.CLK exists | Reading...
GFZ0MGXRAP_20211190000_01D_30S_CLK.CLK file is read in 1.22 seconds
SP3 interpolation is done in 5.20 seconds


C1            L1           D1         S1  \
Epoch               SV                                                        
2021-04-28 21:07:00 C06           NaN  2.243250e+08 -1539.026429  38.000000   
                    C09           NaN  2.125070e+08 -1076.832857  41.000000   
                    C11           NaN  1.352152e+08 -3555.085714  43.428571   
                    C14           NaN  1.240611e+08    32.760429  45.000000   
                    C16           NaN  2.222891e+08 -1595.855857  38.000000   
...                               ...           ...          ...        ...   
2021-04-28 22:07:00 R18  2.326705e+07  1.242018e+08 -3848.505125  46.375000   
                    R19  2.237872e+07  1.197111e+08   -52.467125  37.375000   
                    R20  2.596147e+07  1.388277e+08  3565.564875  37.875000   
                    S23  4.111048e+07  2.160370e+08  -515.803000  44.750000   
                    S36  4.078531e+07  2.143283e+08  -516.305375  41.750000   

                                   C2            L2           D2         S2  \
Epoch               SV                                                        
2021-04-28 21:07:00 C06  4.307927e+07  2.243250e+08 -1539.026429  38.000000   
                    C09  4.080972e+07  2.125070e+08 -1076.832857  41.000000   
                    C11  2.596665e+07  1.352152e+08 -3555.085714  43.428571   
                    C14  2.382462e+07  1.240611e+08    32.760429  45.000000   
                    C16  4.268828e+07  2.222891e+08 -1595.855857  38.000000   
...                               ...           ...          ...        ...   
2021-04-28 22:07:00 R18  2.326705e+07  9.660093e+07 -2993.250750  42.000000   
                    R19  2.237872e+07  9.310873e+07   -40.688500  38.875000   
                    R20  2.596148e+07  1.079770e+08  2773.281125  34.750000   
                    S23           NaN           NaN          NaN        NaN   
                    S36           NaN           NaN          NaN        NaN   

                                   C7            L7           D7         S7  \
Epoch               SV                                                        
2021-04-28 21:07:00 C06  4.307927e+07  1.734648e+08 -1189.841571  31.000000   
                    C09  4.080973e+07  1.643239e+08  -832.746571  36.000000   
                    C11  2.596666e+07  1.045570e+08 -2748.967571  41.000000   
                    C14  2.382462e+07  9.593200e+07    25.282429  42.285714   
                    C16  4.268830e+07  1.718881e+08 -1234.117000  33.000000   
...                               ...           ...          ...        ...   
2021-04-28 22:07:00 R18           NaN           NaN          NaN        NaN   
                    R19           NaN           NaN          NaN        NaN   
                    R20           NaN           NaN          NaN        NaN   
                    S23           NaN           NaN          NaN        NaN   
                    S36           NaN           NaN          NaN        NaN   

                            Azimuth  Elevation  
Epoch               SV                          
2021-04-28 21:07:00 C06   36.614495  10.132689  
                    C09   49.034472  32.742503  
                    C11  177.234549  35.079888  
                    C14  -96.373353  76.785189  
                    C16   38.266603  15.249211  
...                             ...        ...  
2021-04-28 22:07:00 R18 -167.160718  45.315939  
                    R19  -74.711561  63.385367  
                    R20  -26.146660  13.545577  
                    S23         NaN        NaN  
                    S36         NaN        NaN  

[7550 rows x 14 columns]

Orbit and clock files are not downloaded again if they already exist. There are now less rows in the data frame.

## Batch processing
We now use the preprocessing function to process many files and save the outputs as NetCDF files (instead of returning as objects). If we were to process several hundreds of files, the system would likely not have sufficient memory to hold all of the outputs, so it makes sense to processed data as a NetCDF file.

### Specifying several groups of files
Instead of specifying just one file, we use the dictionary to specify a pattern. All files matching the pattern will be processed. We can process several groups files by specifying different matching patterns (see below).

### Specifying where to save data
Same as for specifying the inputs, we use a dictionary to indicate where to save data. The function will create the destination folder if it does not exist.

### Specifying a list of variables to save
For calculating GNSS-VOD, we only need the "S" variables. We can reduce the size of the saved NetCDF files by discarding the other variables, this is done with the 'keepvars' argument, which will only keep the variables present in the passed list. This argument supports UNIX-style pattern matching (e.g. 'S*' will match all variables starting with 'S')

### Compression
Unless `compress=False` is passed as argument, `gv.preprocess()` will compress all S* variables, as well as Azimuth and Elevation when saving to NetCDF. These variables are encoded as Int16 with a scale factor of 0.1. The decoding is automatically applied when reading the data with xarray.

In [7]:
# use gnssvod to batch process the observation RINEX files 
# (files with extension .yyO for each station)
# pattern = {'choice_of_name_for_station1':'pattern to match (UNIX-style)',
#            'choice_of_name_for_station2':'pattern to match (UNIX-style)',
#             ...}
#
pattern = {'Dav2_Twr':'data_RINEX2.11/Dav2_Twr/rinex/*.*O',
          'Dav1_Grnd':'data_RINEX2.11/Dav1_Grnd/rinex/*.*O'}
outputdir = {'Dav2_Twr':'data_RINEX2.11/Dav2_Twr/nc/',
            'Dav1_Grnd':'data_RINEX2.11/Dav1_Grnd/nc/'}
# what variables should be kept
keepvars = ['S?','S??']

gv.preprocess(pattern,interval='15S',keepvars=keepvars,outputdir=outputdir)

/home/huvincen/Work/Projects/GNSS/gnssvod/gnssvod/io/preprocess.py:245: UserWarning: Could not find any files matching the pattern data_RINEX2.11/Dav2_Twr/nc/*.nc
  warnings.warn(f"Could not find any files matching the pattern {search_pattern}")


data_RINEX2.11/Dav2_Twr/rinex/Reach_Dav2_Twr-raw_202104290006.21O exists | Reading...
Observation file  data_RINEX2.11/Dav2_Twr/rinex/Reach_Dav2_Twr-raw_202104290006.21O  is read in 2.42 seconds.
Processing 109702 individual observations
Calculating Azimuth and Elevation
GFZ0MGXRAP_20211180000_01D_05M_ORB.SP3 exists | Reading...
GFZ0MGXRAP_20211180000_01D_05M_ORB.SP3 file is read in 0.27 seconds
GFZ0MGXRAP_20211190000_01D_05M_ORB.SP3 exists | Reading...
GFZ0MGXRAP_20211190000_01D_05M_ORB.SP3 file is read in 0.28 seconds
GFZ0MGXRAP_20211180000_01D_30S_CLK.CLK exists | Reading...
GFZ0MGXRAP_20211180000_01D_30S_CLK.CLK file is read in 1.77 seconds
GFZ0MGXRAP_20211190000_01D_30S_CLK.CLK exists | Reading...
GFZ0MGXRAP_20211190000_01D_30S_CLK.CLK file is read in 2.06 seconds
SP3 interpolation is done in 5.37 seconds
Saved 7370 individual observations in Reach_Dav2_Twr-raw_202104290006.nc
data_RINEX2.11/Dav2_Twr/rinex/Reach_Dav2_Twr-raw_202104290106.21O exists | Reading...
Observation file  d

/home/huvincen/Work/Projects/GNSS/gnssvod/gnssvod/io/preprocess.py:245: UserWarning: Could not find any files matching the pattern data_RINEX2.11/Dav1_Grnd/nc/*.nc
  warnings.warn(f"Could not find any files matching the pattern {search_pattern}")


Observation file  data_RINEX2.11/Dav1_Grnd/rinex/Reach_Dav1_Grnd-raw_202104290006.21O  is read in 2.44 seconds.
Processing 110793 individual observations
Calculating Azimuth and Elevation
GFZ0MGXRAP_20211180000_01D_05M_ORB.SP3 exists | Reading...
GFZ0MGXRAP_20211180000_01D_05M_ORB.SP3 file is read in 0.43 seconds
GFZ0MGXRAP_20211190000_01D_05M_ORB.SP3 exists | Reading...
GFZ0MGXRAP_20211190000_01D_05M_ORB.SP3 file is read in 0.28 seconds
GFZ0MGXRAP_20211180000_01D_30S_CLK.CLK exists | Reading...
GFZ0MGXRAP_20211180000_01D_30S_CLK.CLK file is read in 1.74 seconds
GFZ0MGXRAP_20211190000_01D_30S_CLK.CLK exists | Reading...
GFZ0MGXRAP_20211190000_01D_30S_CLK.CLK file is read in 1.72 seconds
SP3 interpolation is done in 5.25 seconds
Saved 7493 individual observations in Reach_Dav1_Grnd-raw_202104290006.nc
data_RINEX2.11/Dav1_Grnd/rinex/Reach_Dav1_Grnd-raw_202104290106.21O exists | Reading...
Observation file  data_RINEX2.11/Dav1_Grnd/rinex/Reach_Dav1_Grnd-raw_202104290106.21O  is read in 2.

### Skipping existing files by default
The preprocess function will scan the destination folder for existing NetCDF files. If some files are found that have already been processed, these files will be skipped unless overwrite=True has been passed.

Here because the destination folder was empty, a user warning appears in the log above but can be ignored ("Could not find any files matching the pattern data_RINEX2.11/Dav2_Twr/nc/*.nc")